In [1304]:
import xlwings as xw
import pandas as pd
import numpy as np
import re
from pathlib import Path
from dataclasses import (
    dataclass,
    field,
)
from datetime import (
    datetime as dt,
)
from openpyxl.utils import (
    get_column_letter as col_let,
)
from typing import (
    List,
    Dict,
    TypeVar,
    Callable,
)

In [1066]:
deskPath = Path.home() / 'Desktop'
deskPath

WindowsPath('C:/Users/424595/Desktop')

## TypeVars

In [1067]:
xwBook = TypeVar(xw.main.Book)
xwSheet = TypeVar(xw.main.Sheet)
xwRange = TypeVar(xw.main.Range)
pdDataFrame = TypeVar(pd.core.frame.DataFrame)

## Automating Book Selections from Open Books

#### Demonstrative DT Function

In [1068]:
dt.strptime('2019 January', '%Y %B') > dt.strptime('2019 December', '%Y %B')

False

### AFV Sheets

In [1069]:
def redefine_sheets(book: xwBook = xw.books.active) -> Dict[str, xwSheet]:
    '''Creates a dictionary object of the book's sheets.'''
    return {s.name: s for s in book.sheets}

In [1070]:
newAFV = redefine_sheets()

In [1071]:
tots = newAFV['Annual Totals']

In [1072]:
@dataclass
class SheetCorners:
    '''TODO: add default values that calculate start positions'''
    sheet: xwSheet = field(repr=False)
    down: xwRange = field(init=False)
    right: xwRange = field(init=False)
        
    def __post_init__(self):
        self.down = self.sheet['A1048576'].end('up')
        self.right = self.sheet['XFD1'].end('left')
        self.downright = self.right.offset(self.down.row - 1, 0)
        self.entire_range = self.sheet['A1:' + self.downright.address]
    

# Flipping the Totals

In [1073]:
# tots.book.sheets.add('Flipped Totals')

In [1074]:
newAFV = redefine_sheets()

In [1075]:
# flip = newAFV['Flipped Totals']

In [1076]:
# tots[SheetCorners(tots).entire_range.address].api.Copy()
# newAFV['Flipped Totals']['A1'].api.PasteSpecial(
#     Paste=-4163,
#     Operation=-4142,
#     SkipBlanks=False,
#     Transpose=True,
# )

In [1077]:
def _delete_flip():
    tots.book.app.display_alerts = False
    flip.api.Delete()
    tots.book.app.display_alerts = True    

### Merging/Unmerging

In [1078]:
# flip['D1:A1'].api.MergeCells = True
# flip['D2:A2'].api.MergeCells = True
# flip['D3:A3'].api.MergeCells = True

# PD'd

In [1079]:
# tots.book.api.SaveAs(str(deskPath / 'DropDir' / tots.book.name))

In [1080]:
# flipDF = pd.read_excel(
#     tots.book.fullname,
#     header=[0],
#     index_col=[0, 1, 2, 3],
#     sheet_name='Flipped Totals'
# )

In [1081]:
flipDF = pd.DataFrame(
    tots[SheetCorners(tots).entire_range.address].value
).transpose()

### Cols Renamed

In [1082]:
def flip_cols():
    '''Computes the columns in flipDF with a cleaned first row.'''
    return flipDF.iloc[0, :].fillna(
        {i: 'LEVEL_' + str(i) for i in range(4)}
    ).replace(
        {'Year': 'LEVEL_3'}
    )        

In [1083]:
flipDF.rename(columns=flip_cols(), inplace=True)

In [1084]:
flipDF.drop(0, inplace=True)

In [1085]:
flipDF['CLEAN_3'] = flipDF['LEVEL_3']
flipDF['CLEAN_3'].fillna(
    flipDF['LEVEL_2'],
    inplace=True,
)
flipDF['CLEAN_3'].fillna(
    flipDF['LEVEL_1'],
    inplace=True
)

In [1086]:
flipDF['CLEAN_2'] = flipDF['LEVEL_2']
flipDF['CLEAN_2'].fillna(
    flipDF['LEVEL_1'],
    inplace=True,
)
flipDF['CLEAN_2'].fillna(
    method='ffill',
    inplace=True,
)

In [1087]:
flipDF['CLEAN_1'] = flipDF['LEVEL_1']
flipDF['CLEAN_1'].fillna(
    method='ffill',
    inplace=True,
)
flipDF['CLEAN_0'] = flipDF['LEVEL_0']
flipDF['CLEAN_0'].fillna(
    method='ffill',
    inplace=True,
)

### More NaNs

In [1088]:
flipDF['CLEAN_0'].fillna('(blank)', inplace=True)

In [1089]:
flipDF['CLEAN_1'].fillna('(blank)', inplace=True)

In [1090]:
flipDF['CLEAN_2'].fillna('(blank)', inplace=True)

### Cleaning/Filling Data

In [1091]:
flipDF.drop(['TOTAL'], axis='columns', inplace=True)

In [1092]:
# to lower case, strip out spaces, and delimit with undies
for i in range(4):
    flipDF['CLEAN_' + str(i)] = flipDF['CLEAN_' + str(i)].apply(
        lambda x: '_'.join(x.lower().strip().split())
    )

In [1093]:
# fixing subaru
flipDF['CLEAN_2'] = flipDF['CLEAN_2'].apply(
    lambda x: x if 'subaru' not in x else 'subaru'
)

### Renaming PT Type

In [1094]:
flipDF['CLEAN_0'] = flipDF['CLEAN_0'].apply(
    lambda x: x
            .replace('fuel_cell_vehicles', 'fcev')
            .replace('plug-in_hybrids', 'phev')
            .replace('hybrids', 'hev')
            .replace('electric_vehicles', 'bev')
)

### CLEANING PHEV

In [1095]:
flipDF.loc[
    (flipDF['CLEAN_0'] == 'phev')
    & (~flipDF['CLEAN_2'].isin(['non-fleet','fleet','total_us_(no_hawaii)','hawaii',]))
    & (flipDF['CLEAN_2'] == 'total_us_prius_plug-in'),
    'CLEAN_1'
] = 'toyota'

In [1096]:
flipDF.loc[
    (flipDF['CLEAN_0'] == 'phev')
    & (~flipDF['CLEAN_2'].isin(['non-fleet','fleet','total_us_(no_hawaii)','hawaii',]))
    & (flipDF['CLEAN_2'] != 'total_us_prius_plug-in')
    & (
        (~flipDF['CLEAN_2'].str.contains('sales'))
        & (~flipDF['CLEAN_2'].str.contains('share'))
    ),
    'CLEAN_1'
] = 'competitor'

### CLEANING BEV

In [1097]:
flipDF.loc[
    (flipDF['CLEAN_0'] == 'bev')
    & (flipDF['CLEAN_2'] == 'toyota'),
    'CLEAN_1'
] = 'toyota'

In [1098]:
flipDF.loc[
    (flipDF['CLEAN_0'] == 'bev')
    & (flipDF['CLEAN_2'] != 'toyota')
    & (flipDF['LEVEL_1'].isna()),
    'CLEAN_1',
] = 'competitor'

### Renaming toyota_/_lexus

In [1099]:
flipDF['CLEAN_1'] = flipDF['CLEAN_1'].apply(
    lambda x: x if x != 'toyota_/_lexus' else 'toyota'
)

In [1100]:
# fcv toyota is on level 1, nameplate on 3
# hybrid toyota is on level 3
# phev toyota is on level 2
# phev only has 3 levels; toyota on 2, and rav4 ev on 3

# 5 DFs

In [1101]:
fcev = flipDF[
    (flipDF['CLEAN_0'] == 'fcev')
    & (
        # filtering out anything but models
        (flipDF['CLEAN_1'] == 'toyota')
        | (flipDF['CLEAN_1'] == 'competition')
    )
    & (
        (flipDF['CLEAN_3'] == 'total_us')
        | (flipDF['CLEAN_2'] != 'mirai')
    )
].copy()

In [1102]:
fcev.drop(['LEVEL_0', 'LEVEL_1', 'LEVEL_2', 'LEVEL_3'], axis=1, inplace=True)

In [1103]:
fcev.rename(columns={
    'CLEAN_0': 'PT_TYPE',
    'CLEAN_1': 'TOYO/COMP',
    'CLEAN_2': 'BRAND',
    'CLEAN_3': 'NAMEPLATE',
    },
    inplace=True,
)

In [1104]:
hev = flipDF[
    (flipDF['CLEAN_0'] == 'hev')
    & (
        (flipDF['CLEAN_1'] == 'toyota')
        | (flipDF['CLEAN_1'] == 'competition')
    )
    & (
        (flipDF['CLEAN_3'].str.contains('total_us'))
        & (~flipDF['CLEAN_3'].str.contains('hawaii'))
        & (flipDF['CLEAN_3'] != 'total_us_prius_family')
        | (flipDF['CLEAN_1'] != 'toyota')
    )
].copy()

In [1105]:
hev.drop(['LEVEL_0', 'LEVEL_1', 'LEVEL_2', 'LEVEL_3'], axis=1, inplace=True)

In [1106]:
hev.rename(columns={
    'CLEAN_0': 'PT_TYPE',
    'CLEAN_1': 'TOYO/COMP',
    'CLEAN_2': 'BRAND',
    'CLEAN_3': 'NAMEPLATE',
    },
    inplace=True,
)

In [1107]:
phev = flipDF[
    (flipDF['CLEAN_0'] == 'phev')
    & (
        (flipDF['CLEAN_1'] == 'toyota')
        | (flipDF['CLEAN_1'] == 'competitor')
    )
].copy()

In [1108]:
phev.drop(['LEVEL_0', 'LEVEL_1', 'LEVEL_2', 'LEVEL_3'], axis=1, inplace=True)

In [1109]:
phev.rename(columns={
    'CLEAN_0': 'PT_TYPE',
    'CLEAN_1': 'TOYO/COMP',
    'CLEAN_2': 'BRAND',
    'CLEAN_3': 'NAMEPLATE',
    },
    inplace=True,
)

In [1110]:
bev = flipDF[
    (flipDF['CLEAN_0'] == 'bev')
    & (
        (flipDF['CLEAN_1'] == 'toyota')
        | (flipDF['CLEAN_1'] == 'competitor')
    )
].copy()

In [1111]:
bev.drop(['LEVEL_0', 'LEVEL_1', 'LEVEL_2', 'LEVEL_3'], axis=1, inplace=True)

In [1112]:
bev.rename(columns={
    'CLEAN_0': 'PT_TYPE',
    'CLEAN_1': 'TOYO/COMP',
    'CLEAN_2': 'BRAND',
    'CLEAN_3': 'NAMEPLATE',
    },
    inplace=True,
)

In [1113]:
sales = flipDF[
    (~flipDF['CLEAN_3'].str.contains('cumulative'))
    & (flipDF['CLEAN_3'].str.contains('sales'))
    & (~flipDF['CLEAN_3'].str.contains('share'))
    & (
        (~flipDF['LEVEL_1'].isna())
        | (flipDF['CLEAN_2'] == '(blank)')
    )
].copy()

In [1114]:
sales.drop(
    ['LEVEL_0',
     'LEVEL_1',
     'LEVEL_2',
     'LEVEL_3',
     'CLEAN_0',
     'CLEAN_1',
     'CLEAN_2'],
    axis=1,
    inplace=True,
)

In [1115]:
sales.rename(columns={'CLEAN_3': 'SALES'}, inplace=True)

In [1116]:
# reordering columns
sales.set_index(
    sales['SALES'].apply(lambda x: x.replace('_', ' ').upper()),
    inplace=True
)

In [1117]:
sales.drop('SALES', axis=1,inplace=True)

In [1118]:
sales.drop(
    'ALT-FUEL VEHICLE SALES',
    axis='index',
    inplace=True,
)

In [1119]:
sales.fillna(0, inplace=True)

# Add Percentages to OEM DFs

In [1120]:
def group_brands(pt_type_frame: pdDataFrame) -> pdDataFrame:
    ''''''
    oreo = (
        pt_type_frame
        .set_index(['TOYO/COMP', 'BRAND'])
        .loc[:, [c for c in hev.columns if 'CY ' in c and int(c[3:]) > 2009]]
    ).copy().reset_index()
    oreo['OEM'] = (
        oreo['TOYO/COMP'] + '|' + oreo['BRAND']
    ).apply(lambda x: 'toyota' if 'toyota|' in x else x.split('|')[-1])
    
    oreo.drop(['BRAND', 'TOYO/COMP'], axis=1, inplace=True)
    
    return oreo.groupby(oreo['OEM'].apply(
        lambda x: x.replace('_', ' ').upper()
    )).sum().drop(['OEM'], axis=1).copy()


In [1121]:
group_brands(hev)

,CY 2010,CY 2011,CY 2012,CY 2013,CY 2014,CY 2015,CY 2016,CY 2017,CY 2018,CY 2019,CY 2020
OEM,,,,,,,,,,,
BMW,248.0,43.0,406.0,905.0,151.0,30.0,0.0,0.0,0.0,0.0,0.0
FIAT CHRYSLER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FORD,35496.0,27114.0,32543.0,78284.0,64620.0,47261.0,60785.0,90191.0,70409.0,57137.0,9015.0
GM,6705.0,5134.0,33979.0,25066.0,9765.0,4588.0,5182.0,11832.0,8901.0,2477.0,47.0
HONDA,33547.0,31582.0,18166.0,19165.0,26982.0,20914.0,12777.0,24878.0,32534.0,50083.0,6266.0
HYUNDAI,0.0,25918.0,56366.0,41845.0,34828.0,31400.0,25103.0,55604.0,50026.0,37164.0,4030.0
MB,1721.0,310.0,141.0,357.0,188.0,64.0,9.0,7.0,0.0,0.0,0.0
NISSAN,6710.0,3614.0,794.0,1792.0,7794.0,8789.0,4046.0,1615.0,2734.0,3312.0,449.0
SUBARU,0.0,0.0,0.0,0.0,7926.0,5589.0,2173.0,43.0,0.0,0.0,0.0


In [1122]:
def add_p(grouped_oems: pdDataFrame, pt_type: str) -> pdDataFrame:
    ''''''
    pt_type = pt_type.upper()
    for col in grouped_oems:
        grouped_oems[col + ' US ' + pt_type + ' SALES %'] = (
            (grouped_oems[col] / grouped_oems[col].sum()) * 100
        )

    return grouped_oems

In [1123]:
def add_sales_p(sales_frame: pdDataFrame) -> pdDataFrame:
    oreo = sales_frame.loc[:, [c for c in sales_frame.columns if int(c[3:]) > 2009]]
    for col in oreo:
        oreo[col + ' US MARKET SHARE'] = (
            (oreo.loc[[i for i in oreo.index if 'INDUSTRY' not in i],
            col] / oreo.loc['INDUSTRY SALES', col]) * 100
        )
    return oreo

In [1124]:
def gen_plots(p_frame: pdDataFrame) -> Dict[str, goPie]:
    '''Doesn't work. ''' 
    p_series = {}
    labels = tuple(
        map(
            lambda x: x.upper().replace('_', ' '),
                       p_frame.index.to_list()
        )
    )
    
    for p in [p for p in p_frame.columns if '%' in p]:
        p_series.update(
            {
                ' '.join(p.split()[:2]): 
                go.Pie(labels=labels, values=p_frame[p]),
            }
        )

# to Excel

In [1125]:
try:
    tots.book.sheets.add('EVC-AS Charting')
except ValueError as e:
    print(e)
    pass

Sheet named 'EVC-AS Charting' already present in workbook


In [1126]:
newAFV = redefine_sheets()

In [1127]:
evc = newAFV['EVC-AS Charting']

In [1128]:
evc.api.Tab.Color = xw.constants.RgbColor.rgbDarkCyan

In [1129]:
evc.activate()

In [1130]:
evc.clear()

In [1131]:
evc['C1'].value = add_sales_p(sales)

In [1132]:
def reoffset_2() -> xwRange:
    return evc['C1048576'].end('up').offset(2, 0)

In [1133]:
bev.iloc[0, -1]

'bev'

In [1134]:
add_p(group_brands(hev), hev.iloc[0, -1])

,CY 2010,CY 2011,CY 2012,CY 2013,CY 2014,CY 2015,CY 2016,CY 2017,CY 2018,CY 2019,...,CY 2011 US HEV SALES %,CY 2012 US HEV SALES %,CY 2013 US HEV SALES %,CY 2014 US HEV SALES %,CY 2015 US HEV SALES %,CY 2016 US HEV SALES %,CY 2017 US HEV SALES %,CY 2018 US HEV SALES %,CY 2019 US HEV SALES %,CY 2020 US HEV SALES %
OEM,,,,,,,,,,,,,,,,,,,,,
BMW,248.0,43.0,406.0,905.0,151.0,30.0,0.0,0.0,0.0,0.0,...,0.015680,0.088505,0.178725,0.033388,0.007766,0.000000,0.000000,0.000000,0.000000,0.000000
FIAT CHRYSLER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FORD,35496.0,27114.0,32543.0,78284.0,64620.0,47261.0,60785.0,90191.0,70409.0,57137.0,...,9.887105,7.094120,15.460025,14.288210,12.234496,17.134071,23.168135,20.173516,14.231132,15.069707
GM,6705.0,5134.0,33979.0,25066.0,9765.0,4588.0,5182.0,11832.0,8901.0,2477.0,...,1.872110,7.407157,4.950194,2.159151,1.187699,1.460702,3.039387,2.550306,0.616947,0.078566
HONDA,33547.0,31582.0,18166.0,19165.0,26982.0,20914.0,12777.0,24878.0,32534.0,50083.0,...,11.516358,3.960046,3.784827,5.966024,5.414025,3.601580,6.390625,9.321609,12.474190,10.474407
HYUNDAI,0.0,25918.0,56366.0,41845.0,34828.0,31400.0,25103.0,55604.0,50026.0,37164.0,...,9.450984,12.287349,8.263818,7.700863,8.128545,7.076031,14.283476,14.333399,9.256450,6.736652
MB,1721.0,310.0,141.0,357.0,188.0,64.0,9.0,7.0,0.0,0.0,...,0.113041,0.030737,0.070503,0.041569,0.016568,0.002537,0.001798,0.000000,0.000000,0.000000
NISSAN,6710.0,3614.0,794.0,1792.0,7794.0,8789.0,4046.0,1615.0,2734.0,3312.0,...,1.317843,0.173086,0.353896,1.723341,2.275216,1.140486,0.414859,0.783343,0.824921,0.750560
SUBARU,0.0,0.0,0.0,0.0,7926.0,5589.0,2173.0,43.0,0.0,0.0,...,0.000000,0.000000,0.000000,1.752528,1.446829,0.612525,0.011046,0.000000,0.000000,0.000000


In [1135]:
for df in [hev, phev, bev]:
    reoffset_2().value = add_p(group_brands(df), df.iloc[0, -1])

## Charting

In [1354]:
int(dt.now().year - 2010)

10

In [1137]:
# four top right corners
ind_end = evc['XFD1'].end('left')
hev_end = ind_end.offset(7, 0)
phev_end = hev_end.end('down').offset(2, 0)
bev_end = phev_end.end('down').offset(2, 0)

In [1138]:
hev_end.end('left').value = 'HEV'
phev_end.end('left').value = 'PHEV'
bev_end.end('left').value = 'BEV'

In [1139]:
hev_end.address

'$Y$8'

In [1140]:
phev_end.address

'$Y$21'

In [1141]:
bev_end.address

'$Y$35'

In [1347]:
def add_pie_by_column(column): pass  # TODO:

In [1351]:
def delete_labels_less_than_1(pie_chart):
    for lab in pie_chart.Chart.SeriesCollection(1).DataLabels():
        if float(re.findall(r'\d+', lab.Text)[0]) < 0.2:
            lab.Delete()

In [1355]:
for i, right in enumerate([hev_end, phev_end, bev_end]):
    evc.charts.add()
    evc.charts[i].left = 670
    evc.charts[i].top = 75 + (i * 200)
    
    evc.charts[i].set_source_data(
        evc[
            right.address + ':' + right.end('down').address
            + ', ' + 'C' + str(right.row) + ':C' + str(right.end('down').row)
        ]
    )
    
    evc.charts[i].chart_type = 'pie'
    
    pie = evc.api.ChartObjects(evc.charts[i].name)
    
    title = pie.Chart.ChartTitle
    title.Text += ' BY OEM, '
    title.Text = ' '.join(title.Text.split()[2:]) + ' ' + ' '.join(title.Text.split()[:2])
    
    pie.Chart.ApplyDataLabels(
        Type=5,
        LegendKey=True,
        AutoText = True,
        HasLeaderLines=False,
    )
    
    delete_labels_less_than_1(pie)
    pie.Chart.SeriesCollection(1).DataLabels().Position = 2
    
    pie.Chart.HasLegend = False

## Get Gas Data

In [1357]:
ind_end.end('left').end('down').offset(1, 0).value = 'U.S. REGULAR CONVENTIONAL RETAIL GASOLINE PRICE USD/GALLON*'